In [15]:
import csv
import numpy as np
import torch
from torch import optim
import random 
from pytorch_transformers.tokenization_distilbert import DistilBertTokenizer
from pytorch_transformers.modeling_distilbert import DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
special_tokens_dict = {'additional_special_tokens': ['<PLH>', '<s>','[END]']}
tokenizer.add_special_tokens(special_tokens_dict)
encoder = DistilBertModel.from_pretrained('distilbert-base-uncased').cuda()
encoder.resize_token_embeddings(len(tokenizer))

for param in encoder.parameters():
    param.requires_grad = False

pad_token = tokenizer.pad_token
train_data = []
test_data = []
    
with open("memory_conversation.tsv") as csvDataFile:
    csvreader = csv.reader(csvDataFile, delimiter='\t')
    next(csvreader)
    
    data = []
    conversation = {
                "conv_id":None,
                "exchange_ids":[],
                "user_responses":[],
                "num_states":1,
                "state_updates":[],
                "agent_utterances":[],
                "state_embeddings":[]
            }
    max_states = 0
    max_utterances = 0
    
    for conv_id, exchange_id, agent, user, action1, action2, action3 in csvreader:
        if conversation["conv_id"] is None:
            conversation["conv_id"] = conv_id
        elif conv_id != conversation["conv_id"]:
            data.append(conversation)
            max_utterances = max(
                max_utterances, 
                len(conversation["exchange_ids"])
            )
            conversation = {
                "conv_id":conv_id,
                "exchange_ids":[],
                "user_responses":[],
                "num_states":1,
                "state_updates":[],
                "agent_utterances":[],
                "state_embeddings":[]
            }
        
        conversation["agent_utterances"].append(agent)
        conversation["user_responses"].append(user)
        
        state_updates = []
        for action in (action1, action2, action3):
            if action != "NULL":
                tokenized_action = torch.cuda.LongTensor([
                    tokenizer.encode(
                        action.split(", ")[1]
                    )
                ])
                embedding = encoder(tokenized_action)[0]
                idx = conversation["num_states"]
                conversation["num_states"] += 1
                state_updates.append((idx, embedding))
        conversation["state_updates"] = state_updates
        conversation["exchange_ids"].append(exchange_id)
        max_states = max(max_states, len(conversation["exchange_ids"]))
    if len(conversation) > 0:
        data.append(conversation)
    max_utterances = max(
        max_utterances, 
        len(conversation["agent_utterances"])
    )

In [32]:
class Decoder(torch.nn.Module):
    def __init__(self, vsz, esz=768, hsz=768, num_layers=8):
        super().__init__()
        self.attn = []
        self.dense = []
        self.relu = []
        self.num_layers = num_layers
        self.norm = []
        
        for i in range(num_layers):
            if i == 0:
                self.attn.append(
                    torch.nn.MultiheadAttention(esz, 8).cuda()
                )
                self.dense.append(
                    torch.nn.Linear(
                        esz, hsz,
                    ).cuda()
                )
            else:
                self.attn.append(
                    torch.nn.MultiheadAttention(hsz, 8).cuda()
                )
                self.dense.append(
                    torch.nn.Linear(
                        hsz, hsz,
                    ).cuda()
                )
            self.relu.append(torch.nn.ReLU().cuda())
            self.norm.append(torch.nn.LayerNorm((300,1,hsz)).cuda())
        self.out = torch.nn.Linear(
            hsz, vsz,
        ).cuda()
        
    # accepts seq_len x num_values  returns num_keys
    def forward(self, input, hidden=None):
        input, mask = input
        for i in range(self.num_layers):
            attn_mask, _ = self.attn[i](input, input, input, key_padding_mask=mask)
            input_a = attn_mask + input
            input_b = self.relu[i](self.dense[i](input_a))
            input = self.norm[i](input_b)
        
        return self.out(input)

In [33]:
class Model():
    def __init__(self, lr=0.0005, hsz=768, pad_len=300):
        self.pad_len = pad_len
        self.decoder = Decoder(len(tokenizer)).cuda()
        self.optims = {
            'decoder': optim.Adam(self.decoder.parameters(), lr=lr),
        }
        self.loss = 0
        self.step = 0
        loss_weights = torch.ones(len(tokenizer))
        loss_weights[tokenizer.encode("[END]")[0]] = 2
        self.criterion = torch.nn.CrossEntropyLoss(ignore_index=0)
    
    def zero_grad(self):
        for optimizer in self.optims.values():
            optimizer.zero_grad()
            
    def update_params(self):
        for optimizer in self.optims.values():
            optimizer.step()
    
    def encode_option(self, i, conversation, dialog):
        prefix = conversation["exchange_ids"][i][:-1]
        ids, utterances, responses = [], [], []
        level = conversation["exchange_ids"][i].count("_")
        while conversation["exchange_ids"][i].count("_") == level:
            utterances.append(conversation["agent_utterances"][i])
            responses.append(conversation["user_responses"][i])
            ids.append(conversation["exchange_ids"][i])
            i += 1

        idx = random.choice(range(len(ids)))
        id = ids[idx]
        dialog.append((utterances[idx], responses[idx]))

        while prefix in conversation["exchange_ids"][i]:
            
            if id not in conversation["exchange_ids"][i]:
                i += 1
                continue

            if conversation["exchange_ids"][i].count("_") == level + 1:
                dialog.append(
                    (conversation["agent_utterances"][i], 
                    conversation["user_responses"][i])
                )
                i += 1
            elif conversation["exchange_ids"][i].count("_") == level + 2:
                i, _ = self.encode_option(i, conversation, dialog)
            else:
                raise Exception()
        return i, dialog
        
    def tokenize(self, text):
        tokens = tokenizer.encode(text)
        while len(tokens) < self.pad_len:
            tokens.append(tokenizer.pad_token_id)
        return torch.cuda.LongTensor([tokens])
        
    def encode(self, text):
        tokens = self.tokenize(text)
        return encoder(tokens)[0], tokens.long()
            
    def generate_turn(self, conversation):
        history = ""
        i = 0
        while i < len(conversation["exchange_ids"]):
            exchange_id = conversation["exchange_ids"][i]
            if not exchange_id.endswith("_A"):
                agent = random.choice(conversation["agent_utterances"][i].split("|"))
                user = conversation["user_responses"][i]
                
                if user == "NULL":
                    if history == "":
                        history += "[AGENT] " + agent
                        continue
                    current_encoded, current_tokens = self.encode(history +  " [END]")
                    history += "[AGENT] " + agent
                    next_encoded, next_tokens = self.encode("[AGENT] " + agent +  " [END]")
                    yield current_encoded, current_tokens, next_tokens
                else:
                    history += "[USER] " + user
                i += 1
            else:
                i, dialog = self.encode_option(i, conversation, [])
                for agent, user in dialog:
                    if user == "NULL":
                        current_encoded, current_tokens = self.encode(history  + " [END]")
                        history += "[AGENT] " + random.choice(agent.split("|"))
                        next_tokens = self.tokenize("[AGENT] " + random.choice(agent.split("|")) + " [END]")
                        yield current_encoded, current_tokens, next_tokens
                    else:
                        history += "[USER] " + random.choice(user.split("|"))
        
    def train_step(self):
        conversation = random.choice(data)
        self.zero_grad()
        
        for current_encoded, current_tokens, prediction_tokens in self.generate_turn(conversation):
            mask = current_tokens == tokenizer.pad_token_id
            current_encoded = current_encoded.transpose(0,1)
            decoded = self.decoder((current_encoded, mask)).transpose(0,1).transpose(1,2)
            loss = self.criterion(decoded, prediction_tokens)
            self.step += 1
            self.loss += loss
            loss.backward()
            self.update_params()
            if self.step % 100 == 0:
                print("Step %d Loss %f" % (self.step, self.loss.item() / 50))
                self.loss = 0
                #self.eval_step()
                #decoded = 
                current_tokens = [t for t in current_tokens[0].tolist() if t != tokenizer.pad_token_id]
                print("Source")
                print(tokenizer.decode(current_tokens))
                print("Target")
                prediction_tokens = [t for t in prediction_tokens[0].tolist() if t != tokenizer.pad_token_id]
                print(tokenizer.decode(prediction_tokens))
                print("Predicted")
                #print(decoded.size())
                predicted_tokens = torch.argmax(decoded, 1)
                #print(predicted_tokens.size())
                predicted_tokens = [t for t  in predicted_tokens[0].tolist() if t != tokenizer.pad_token_id]
                #print(predicted_tokens)
                print(tokenizer.decode(predicted_tokens))
                #print(tokenizer.decode(predicted_tokens)[:500])
        
            
            

model = Model()
for i in range(1000):
    model.train_step()

Step 100 Loss 8.105240
Source
[ agent ] null [ agent ] null [ agent ] good evening! [ user ] hello. [ agent ] i feel like talking about my dad. is that ok? [ user ] sure [ user ] can we talk about your mother? [ agent ] my dad just turned 65 [ agent ] he used to be a doctor, but now he's retired. [ agent ] ok fine. how is your mother doing? [ user ] she is still sick. [ agent ] i'm sorry to hear that. [END]
Target
[ agent ] are you still living in the same house? [END]
Predicted
[ agent ] that ] still ] in'same house [END] [END] [END] [END] [END]. [END] [END] [END] [END]. [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END] [END]you [END] [END] [END] [END] [END] [END] [END]you [END] [END] [END] [END] [END] [END] [END] [END]? [END] [END] [END]you [END] [END] [END]you [END] [END] [END]you s [END] [END]agent [END] [END] [END]]? [END]agent ] [END]? agent ] [END] [END] [END] [END] [E

Step 800 Loss 1.453818
Source
[ agent ] null [ agent ] null [ agent ] hello [ agent ] do you want to talk about sports? [ user ] no, i'd prefer to talk about family [END]
Target
[ agent ] ok, let's talk about family. [END]
Predicted
[ agent ] ok, let's talk about family. [END]about family about. agent about about about. agent [END]. family [END]about. [END]s [END]about about about... [END][ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 900 Loss 1.293694
Source
[ agent ] null [ agent ] null [ agent ] hi! [ agent ] di

Step 1400 Loss 1.024240
Source
[ agent ] null [ agent ] null [ agent ] hello [ agent ] do you want to talk about sports? [ user ] no, i'd prefer to talk about family [ agent ] ok, let's talk about family. [ agent ] how old is your father? [ user ] he is 35. [ agent ] that's so young! [ agent ] how about your mother? [ user ] she is 40. [ agent ] she is also very young. [END]
Target
[ agent ] do you have any siblings? [END]
Predicted
[ agent ] do you have any siblings? [END]. [END] [END] [END]. very [END] [END] [END] [END] [END]? agent [END] [END]. [END]is [END] [END]s [END] [END] [END] [END] [END] [END] [END]. [END]? agent s s [END] [END]have. [END]agent you is agent agent agent agent agent [END]agent you [END]agent is [END] [END]agent. agent s [END] [END] [END] [END] [END]agent you agent agent agent agent agent [END]agent you [END]agent [END] [END] [END]agent any [END] [END] [END] [END]? [END] [END] [END][ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [

Step 2300 Loss 0.907356
Source
[ agent ] null [ agent ] null [ agent ] good evening! [ user ] hello. [ agent ] i feel like talking about my dad. is that ok? [ user ] sure [ user ] can we talk about your mother? [ agent ] my dad just turned 65 [ agent ] he used to be a doctor, but now he's retired. [ agent ] ok fine. how is your mother doing? [ user ] she is still sick. [ agent ] i'm sorry to hear that. [ agent ] are you still living in the same house? [ user ] yes [END]
Target
[ agent ] remind me, how many bathrooms did you have? [END]
Predicted
[ agent ] remind me, how many bathrooms did you have [END] [END] [END]agent you [END] [END] [END]agent how [END] [END] [END] [END]agent [END]agent you you you. agent [END] [END]how [END]how are how [END]s [END]agent is are agent [END]agent s [END]that!! that you that you! you you that you agent to that [END]! do that you [END] [END]did you [END] [END]you agent that agent agent agent agent you are you agent you you s you [END]agent you you that 

Step 3000 Loss 0.419167
Source
[ agent ] null [ agent ] null [ agent ] hi! [ agent ] did you want to talk about your family? [ user ] sure [ agent ] let's start with your father. [ agent ] is your father still alive? [ user ] no, he is dead. [ agent ] i'm sorry to hear that. [ agent ] what about your mother? [ user ] she is still alive. [ agent ] that's good to hear. [ agent ] how old is she this year? [ user ] she is 50 [ agent ] that is quite old [ agent ] how old are you? [ user ] i am 25. [ agent ] you are very young too. [ agent ] do you have any siblings? [ user ] no, i don't have any. [END]
Target
[ agent ] can you describe your house? [END]
Predicted
[ agent ] can you is your siblings? [END]. [END] [END] [END]? you agent [END] [END] [END] [END]agent agent agent agent [END] [END]] ] [END] [END]you that s s [END]? old agent agent [END]agent old agent agent that that you agent [END]agent young agent about that you you s [END]? your [END]so [END] [END] [END]? that agent [END]agent 

...

Step 9500 Loss 0.277199
Source
[ agent ] null [END]
Target
[ agent ] null [END]
Predicted
[ agent ] null [END] [END]null [END] [END][ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [
Step 9600 Loss 0.226770
Source
[ agent ] null [ agent ] null [ agent ] good evening! [ user ] hello. [ agent ] i feel like talking about my dad. is that ok? [ user ] sure [ user ] can we talk about your mother? [ agent ] my dad just turned 65 [ agent ] he used to be a doctor, but now h

Step 10300 Loss 0.543248
Source
[ agent ] null [ agent ] null [ agent ] hi! [ agent ] did you want to talk about your family? [ user ] sure [ agent ] let's start with your father. [ agent ] is your father still alive? [ user ] no, he is dead. [ agent ] i'm sorry to hear that. [ agent ] what about your mother? [ user ] she is still alive. [ agent ] that's good to hear. [ agent ] how old is she this year? [ user ] she is 50 years old. [END]
Target
[ agent ] that is quite old [END]
Predicted
[ agent ] oh is that is very young. [END] [END] [END] [END]? is very is is? have agent agent agent agent [END]that is [END] [END] [END]you that s??? agent you agent is agent is agent is that that that agent [END]agent young is about that you that is [END]? is [END]that that??? you agent [END]? is [END]agent agent agent agent [END]agent is agent you you you is [END]? you [END]that [END] [END] [END] [END]about [END]agent you [END] [END]agent [END]. that ] [END]agent any [END]about [END]about,. [END] [EN

Step 11000 Loss 0.344659
Source
[ agent ] null [ agent ] null [ agent ] hi! [ agent ] did you want to talk about your family? [ user ] sure [ agent ] let's start with your father. [ agent ] is your father still alive? [ user ] no, he is dead. [ agent ] i'm sorry to hear that. [ agent ] what about your mother? [ user ] she is still alive. [ agent ] that's good to hear. [ agent ] how old is she this year? [ user ] she is 40 [ agent ] oh, that is very young. [ agent ] how old are you? [ user ] i am 25. [ agent ] you are very young too. [END]
Target
[ agent ] do you have any siblings? [END]
Predicted
[ agent ] do you have any siblings? [END]any [END] [END] [END]agent you agent have [END] [END] [END]agent agent agent agent [END] [END]you [END] [END] [END]you that s s were? agent have agent [END]agent you agent agent have that you agent [END]agent young agent about that you you s [END]? you [END]that that [END]?? you agent [END]agent you old agent agent agent agent [END]agent young agent you

Step 11700 Loss 0.733681
Source
[ agent ] null [ agent ] null [ agent ] hello [ agent ] do you want to talk about sports? [ user ] no, i'd prefer to talk about family [ agent ] ok, let's talk about family. [ agent ] how old is your father? [ user ] he is 35. [ agent ] that's so young! [ agent ] how about your mother? [ user ] she is 40. [ agent ] she is also very young. [ agent ] do you have any siblings? [ user ] no. [ agent ] you are not an only child then. [ agent ] ah, so you are an only child. [END]
Target
[ agent ] how old was your father when were you were born? [END]
Predicted
[ agent ] how old not you father when then you [END]born then [END]house [END] [END] [END] [END]that? agent [END]?. young about [END]that that? agent [END]that have [END]? you? about agent agent agent?? have agent born agent agent agent is agent agent agent agent [END]agent agent agent agent is is born agent you agent s [END]agent child [END]born agent agent agent agent agent agent agent [END]agent agent 

Step 12400 Loss 0.454648
Source
[ agent ] null [ agent ] null [ agent ] hi! [ agent ] did you want to talk about your family? [ user ] sure [ agent ] let's start with your father. [ agent ] is your father still alive? [ user ] no, he is dead. [ agent ] i'm sorry to hear that. [ agent ] what about your mother? [ user ] she is still alive. [END]
Target
[ agent ] that's good to hear. [END]
Predicted
[ agent ] that's good to hear. [END] [END] [END] [END].'about. about that you agent agent agent agent [END]? is [END] [END].'that s s?? agent that agent is agent agent about about that that that agent [END]agent agent is about that that that agent [END]? is that that that that that? you agent [END]agent agent agent agent agent agent agent [END]agent agent agent that that that.. [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [

In [ ]:
"foooo"[:5] [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ [ 

In [ ]:
data[2]